#### Get the Yelp API endpoint etc...

In [2]:
import requests
import boto3
import os, time, datetime, json
from dotenv import load_dotenv

load_dotenv()

# get the api kep
API_KEY = os.getenv("YELP_API_KEY")
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

if not API_KEY:
    raise ValueError("YELP_API_KEY not found in .env file")

HEADERS = {
    "Authorization": f"Bearer {API_KEY}"
}

# yelp endpoint
URL = "https://api.yelp.com/v3/businesses/search"


#get the restaurants

#### def to get all the data the api

In [3]:

#get the restaurants
def get_restaurants(term, location="New York, NY", total=200):
    restaurants = []
    offset = 0

    print(f"\nGetting data {term}...")
    
    while len(restaurants) < total:
        params = {
            "term": term,
            "location": location,
            "limit": 50,
            "offset": offset
        }

        response = requests.get(URL, headers=HEADERS, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        data = response.json()
        businesses = data.get("businesses", [])

        if not businesses:
            break

        restaurants.extend(businesses)
        offset += 50

        print(f"Collected {len(restaurants)} so far...")

        time.sleep(0.5)  # respect the api....

    return restaurants[:total]


### call the funct with the defined cuisnes

In [ ]:
# API CALLS HERE RUN THIS CELL ONLY WHEN NEEDED USE VAR FROM HERE ON OUT
cuisines = [
    "Chinese restaurants",
    "Italian restaurants",
    "Mexican restaurants",
    "Indian restaurants",
    "Japanese restaurants"
]


all_restaurants = {}

# API CALLS HERE RUN THIS CELL ONLY WHEN NEEDED USE VAR FROM HERE ON OUT
for cuisine in cuisines:
    results = get_restaurants(cuisine)

    print(f"Total pulled for {cuisine}: {len(results)}")

    for r in results:
        all_restaurants[r["id"]] = r  # overwrite prevents duplicates



In [ ]:

print("\n====================================")
print("TOTAL UNIQUE RESTAURANTS:", len(all_restaurants))
print("====================================\n")

### remove duplicates and create json file for fun

In [ ]:
clean_restaurants = []

timestamp = datetime.utcnow().isoformat()

for r in all_restaurants.values():
    cleaned = {
        "businessId": r["id"],
        "name": r["name"],
        "address": " ".join(r["location"]["display_address"]),
        "coordinates": r["coordinates"],
        "numReviews": r["review_count"],
        "rating": r["rating"],
        "zipCode": r["location"].get("zip_code", ""),
        "insertedAtTimestamp": timestamp
    }

    clean_restaurants.append(cleaned)


# -------- Write to JSON File --------
output_file = "yelp_restaurants.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(clean_restaurants, f, indent=4)

print("\n====================================")
print(f"Saved {len(clean_restaurants)} restaurants to {output_file}")
print("====================================")